- 1.读取数据集

In [1]:
import collections
import re
from d2l import torch as d2l

#@save
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt', '090b5e7e70c295757f55df93cb0a180b9691891a')

#@save
def read_time_machine():
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# 文本行数: {len(lines)}')
print(lines[0])
print(lines[10])

# 文本行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


- 2. 词元化

In [3]:
def tokenize(lines, token='word'):
    """将文本行拆分为单词或字符词元"""
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知词元类型：' + token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


- 3. 词表

In [27]:
#@save
class Vocab:
    """文本词表"""
    def __init__(self, tokens=None, min_req=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 按出现评论排序
        counter = count_corpus(tokens) # 统计单词数量 格式 {'word': count}
        print(counter.items()) # counter.items() 将元素有key:value 格式调整为 元组格式(key,value)
        # key=lambda x: x[1] 表示按数量作为key进行排序
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True)

        # 未知词元索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_token)}

        for token, freq in self._token_freqs:
            if freq < min_req:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) -1
        print(self.idx_to_token)
        print(self.token_to_idx)

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        # print("__getitem__", tokens)
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_token(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):
        return 0

    @property
    def token_reqs(self):
        return self._token_freqs

def count_corpus(tokens):
    """统计词元的频率"""
    if len(tokens) == 0 or  isinstance(tokens[0], list):
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]

    return collections.Counter(tokens)


# vocab = Vocab(tokens)
# print(list(vocab.token_to_idx.items())[:10])

# for i in [0, 10]:
#     print('文本:', tokens[i])
#     print('索引:', vocab[tokens[i]])

- 4. 整合所有功能

1. 先读取数据 lines = read_time_machine()
2. 再将数据按照字符分割
3. 将分割后的数据给到 Vocab 类
    3.1 对 tokens 按照词频进行排序
    3.2 创建索引和词元映射
4. 遍历tokens, 获取字符对应的索引
5. 如果指定了 max_tokens 参数，则只返回前 max_tokens 个最频繁的词元

In [28]:
#@save
def load_corpus_time_machine(max_tokens=-1):
    """"返回时光机器数据集的词元索引列表和词表"""
    lines = read_time_machine() # 读取数据
    tokens = tokenize(lines, 'char') # 分词, 将数据分割成词元
    # print(tokens)
    vocab = Vocab(tokens) # 建立词表
    # 因为时光机器数据集中的每个文本行不一定是一个句子或一个段落，
    # 所以将所有文本行展平到一个列表中
    # vocab[token] 调用 Vocab 的 __getitem__ 方法
    corpus = []
    for line in tokens:
        for token in line:
            # print(vocab[token])
            corpus.append(vocab[token]) # 将词元转换成索引
    # corpus = [vocab[token] for line in tokens for token in line] # 将数据展平到列表中
    if max_tokens > 0:
        corpus = corpus[:max_tokens]

    return corpus, vocab

corpus, vocab = load_corpus_time_machine(100)
print(len(corpus))
# print(corpus)
print(len(vocab))
# print(vocab)

dict_items([('t', 13515), ('h', 8257), ('e', 17838), (' ', 29927), ('i', 10138), ('m', 4043), ('a', 11704), ('c', 3424), ('n', 9917), ('b', 1897), ('y', 2679), ('g', 3075), ('w', 3225), ('l', 6146), ('s', 8486), ('r', 7674), ('v', 1295), ('f', 3354), ('o', 9758), ('p', 2427), ('k', 1087), ('x', 236), ('u', 3805), ('d', 6337), ('z', 144), ('j', 97), ('q', 95)])
['<unk>', ' ', 'e', 't', 'a', 'i', 'n', 'o', 's', 'h', 'r', 'd', 'l', 'm', 'u', 'c', 'f', 'w', 'g', 'y', 'p', 'b', 'v', 'k', 'x', 'z', 'j', 'q']
{'<unk>': 0, ' ': 1, 'e': 2, 't': 3, 'a': 4, 'i': 5, 'n': 6, 'o': 7, 's': 8, 'h': 9, 'r': 10, 'd': 11, 'l': 12, 'm': 13, 'u': 14, 'c': 15, 'f': 16, 'w': 17, 'g': 18, 'y': 19, 'p': 20, 'b': 21, 'v': 22, 'k': 23, 'x': 24, 'z': 25, 'j': 26, 'q': 27}
100
[3, 9, 2, 1, 3, 5, 13, 2, 1, 13, 4, 15, 9, 5, 6, 2, 1, 21, 19, 1, 9, 1, 18, 1, 17, 2, 12, 12, 8, 5, 3, 9, 2, 1, 3, 5, 13, 2, 1, 3, 10, 4, 22, 2, 12, 12, 2, 10, 1, 16, 7, 10, 1, 8, 7, 1, 5, 3, 1, 17, 5, 12, 12, 1, 21, 2, 1, 15, 7, 6, 22, 2, 6